### Testing the most efficient model : multilingual-e5-large-instruct (0.5 B)

 Even if this model is ranked at the 5th position it is the lighetest and most efficient one without a significant drop in performance (only 0.5 B param)

In [ ]:
import os
from transformers import AutoTokenizer, AutoModel

In [ ]:
tokenizer= AutoTokenizer.from_pretrained("intfloat/multilingual-e5-large-instruct",token =os.getenv('HF_token'))
model= AutoModel.from_pretrained("intfloat/multilingual-e5-large-instruct", token=os.getenv('HF_token'))

In [9]:
def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'

task = 'Given a web search query, retrieve relevant passages that answer the query'
queries = [
    get_detailed_instruct(task, 'how much protein should a female eat'),
    get_detailed_instruct(task, '南瓜的家常做法')
]
documents = [
    "As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day...",
    "1.清炒南瓜丝 原料:嫩南瓜半个 调料:葱、盐、白糖、鸡精 做法:..."
]
input_texts = queries + documents

In [10]:
batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

In [11]:
import torch

In [12]:
with torch.no_grad():
    outputs = model(**batch_dict)
    embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

Définir la fonction de pooling :
Cette fonction effectue une moyenne pondérée par le masque d’attention.

In [13]:
import torch.nn.functional as F

embeddings = F.normalize(embeddings, p=2, dim=1)

In [14]:
embeddings

tensor([[ 0.0202,  0.0112, -0.0451,  ..., -0.0266, -0.0423,  0.0389],
        [ 0.0486,  0.0594,  0.0040,  ..., -0.0214, -0.0263,  0.0198],
        [ 0.0198, -0.0023, -0.0366,  ..., -0.0368, -0.0272,  0.0254],
        [ 0.0461,  0.0436,  0.0091,  ..., -0.0155, -0.0181,  0.0149]])

In [15]:
similarity = torch.matmul(embeddings[0], embeddings[1].T)

/var/folders/ty/2g_hgfks4nl_84mz496fmn600000gn/T/ipykernel_82697/1230817599.py:1: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3729.)
  similarity = torch.matmul(embeddings[0], embeddings[1].T)


In [16]:
embedding_list = embeddings.tolist()  # Conversion en liste Python

import numpy as np
embedding_array = embeddings.cpu().numpy()  # Conversion en tableau NumPy

In [17]:
embedding_array

array([[ 0.02018598,  0.01120441, -0.04514236, ..., -0.02658733,
        -0.04230955,  0.03893524],
       [ 0.04857629,  0.05938283,  0.00398616, ..., -0.02135831,
        -0.0263372 ,  0.01975931],
       [ 0.01981018, -0.00233509, -0.03661876, ..., -0.03679116,
        -0.02720564,  0.02541652],
       [ 0.04608595,  0.04364255,  0.00913067, ..., -0.01546213,
        -0.01805959,  0.01493509]], dtype=float32)